In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize, RandomHorizontalFlip, RandomRotation
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

In [163]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)



In [164]:
def means_std_dataset(dataset):
    norm_dataset = torch.stack([img_t for img_t, _ in dataset], dim=1)

    means = norm_dataset.view(1, -1).mean(dim=1)
    std = norm_dataset.view(1, -1).std(dim=1)

    return means, std

In [165]:
means_training, std_training = means_std_dataset(training_data)

print(means_training, std_training)

means_test, std_test = means_std_dataset(test_data)

tensor([0.1307]) tensor([0.3081])


In [ ]:
final_transform_training = Compose([
    RandomHorizontalFlip(p=0.5),  # 50% de chance de retourner l'image
    RandomRotation(degrees=15),
    ToTensor(),
    Normalize(means_training, std_training)
])

final_transform_test = Compose([
    ToTensor(),
    Normalize(means_test, std_test)
])


training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=final_transform_training
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=final_transform_test
)

In [167]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [168]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [169]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Conv2d(1, 64, 3, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(p=0.02),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(p=0.02),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        logits = self.fc(x)
        return logits

In [170]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)
model = NeuralNetwork().to(device)
print(model)

cuda
NeuralNetwork(
  (fc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=6272, out_features=512, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.02, inplace=False)
    (8): Linear(in_features=512, out_features=128, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.02, inplace=False)
    (11): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [171]:
x = torch.rand(1, 1,28,28,device=device)
logits = model(x)

In [172]:
logits

tensor([[-0.0690,  0.0884,  0.0282,  0.0456, -0.0142, -0.0493, -0.1240,  0.0986,
         -0.1069,  0.0515]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [173]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0935, 0.1095, 0.1031, 0.1049, 0.0988, 0.0954, 0.0885, 0.1106, 0.0901,
         0.1055]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [174]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([7], device='cuda:0')

In [175]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [176]:
#writer = SummaryWriter()

In [177]:
def train(dataloader, model, loss_fn, optimizer):
    #writer = SummaryWriter()
    size= len(dataloader.dataset)
    model.train()
    losses = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss= loss_fn(pred, y)
        losses += loss
        #writer.add_scalar("Loss/train", loss.item(), batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            #writer.add_scalar("Loss/train", loss.item(), batch)
            loss, current = loss.item(), (batch + 1) * len(X)
            #print(f"loss : {loss:>7f}, [{current:>5d}/{size:>5}]")

    #writer.flush()
    return losses / size

In [178]:
def test(dataloader, model, loss_fn):
    #writer = SummaryWriter()
    size= len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct= 0, 0

    with torch.no_grad():
        for batch,(X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)
            temp_loss = loss_fn(pred, y).item()
            temp_correct = (pred.argmax(1) == y).type(torch.float).sum().item()
            #writer.add_scalar("Loss/test", temp_loss, batch)
            

            test_loss+= temp_loss
            correct+= temp_correct

            #if batch % 10 == 0:
                #writer.add_scalar("Loss/test", temp_loss, batch)
    
    test_loss/=num_batches
    correct/=size

    #writer.flush()
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    return 100*correct



In [179]:
epochs = 3
accs = []
for t in range(epochs):
    #print(f"Epoch {t+1}\n--------------------------------")
    losses = train(train_dataloader, model, loss_fn, optimizer)
    #writer.add_scalar("Loss/Train", losses, t)
    acc = test(test_dataloader, model, loss_fn)
    accs.append(acc)
    
#writer = SummaryWriter()
#for t in range(epochs):
#    writer.add_scalar("Accuracy/test", accs[t], t)

print("Done")
#writer.flush()
#writer.close()

Test Error: 
 Accuracy: 98.5%, Avg loss: 0.044680

Test Error: 
 Accuracy: 98.4%, Avg loss: 0.050800

Test Error: 
 Accuracy: 98.7%, Avg loss: 0.051514

Done


In [180]:
example_inputs = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)

[torch.onnx] Obtain model graph for `NeuralNetwork([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NeuralNetwork([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [181]:
onnx_program.save("my_model.onnx")

In [182]:
import torchvision.models as models

pretrained_model = models.resnet50(pretrained=True)

e:\Cours\Machine Learning\Projet\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Cours\Machine Learning\Projet\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [183]:
class ModifiedResNet(nn.Module):
    def __init__(self):
        super(ModifiedResNet, self).__init__()
        self.resnet = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
        self.resnet.conv1 = nn.Conv2d(
            in_channels=1, 
            out_channels=64, 
            kernel_size=7, 
            stride=2, 
            padding=3, 
            bias=False
        )
        num_classes = 10
        self.resnet.fc = nn.Linear(pretrained_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

model = ModifiedResNet()

Using cache found in C:\Users\Administrator/.cache\torch\hub\pytorch_vision_main


In [184]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)
model = ModifiedResNet().to(device)
print(model)

cuda


Using cache found in C:\Users\Administrator/.cache\torch\hub\pytorch_vision_main


ModifiedResNet(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [185]:
for param in model.parameters():
    param.requires_grad = False

In [186]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [187]:
for param in model.resnet.layer4.parameters():
    param.requires_grad = True

In [188]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done")

Epoch 1
--------------------------------
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.288879

Epoch 2
--------------------------------
Test Error: 
 Accuracy: 92.4%, Avg loss: 0.236865

Epoch 3
--------------------------------
Test Error: 
 Accuracy: 92.1%, Avg loss: 0.240191

Epoch 4
--------------------------------
Test Error: 
 Accuracy: 92.3%, Avg loss: 0.242999

Epoch 5
--------------------------------
Test Error: 
 Accuracy: 92.4%, Avg loss: 0.248751

Done


In [189]:
example_inputs = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)

[torch.onnx] Obtain model graph for `ModifiedResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ModifiedResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 106 of general pattern rewrite rules.


In [190]:
onnx_program.save("custom_resnet_model.onnx")